# 第四课：图神经网络算法（二）

本节教程将带着同学们理解[GraphSage模型](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf)的关键代码，以便按照自己的需求修改和实现。请参照示例代码，补充实现采样函数和不同的聚合函数。

In [1]:
# 安装依赖
# !pip install paddlepaddle==1.8.5
!pip install pgl

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 7.9MB 22.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 34.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.0MB/s eta 0:00:01


## 1. 代码框架梳理

GraphSage的PGL代码实现位于 [PGL/examples/graphsage/](https://github.com/PaddlePaddle/PGL/tree/main/examples/graphsage)，Notebook中提供了复制版本，主要结构如下

- **数据部分** ./data/

	我们在原 github 上使用的是Reddit数据集。Reddit是一个新闻网站，以消息为节点，如果同一用户在不同消息下都发表了评论，则二者之间有边，用于预测消息属于哪类社区。
    
    但是因为数据集比较大，我们暂时没能放入 AIStudio。所以很遗憾的，这里我们不需要使用 Reddit 数据集进行评估，作业针对的 Acc 结果只需在 Cora 数据集上完成即可，这样也方便大家快速跑出结果。感兴趣的同学，自己跑跑 Reddit 啦！这里贴出数据集链接：[reddit.npz](https://drive.google.com/open?id=19SphVl_Oe8SJ1r87Hr5a6znx3nJu1F2J)和[reddit_adj.npz](https://drive.google.com/open?id=174vb0Ws7Vxk_QTUtxqTgDHSQ4El4qDHt)
    
    由于我们将原本可以全图跑的 Cora数据集进行了分 Batch 跑，因此相对的，测试集能够达到的结果就比我们昨天所讲解的 GCN、GAT 要低了。
    
- **采样部分**  reader.py 

	提供了采样代码。以某个节点为中心，按照距该节点的距离依次采样得到子图，作为训练数据
    
- **模型部分** model.py
	
    提供了聚合函数实现，包括Mean，Maxpool，MeanPool和LSTM四种方式
    
- **训练部分** train.py

	实现了数据读取、模型构建和模型训练部分。

## 2. GraphSage采样函数实现

GraphSage的作者提出了采样算法来使得模型能够以Mini-batch的方式进行训练，算法伪代码见[论文](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf)附录A。

- 假设我们要利用中心节点的k阶邻居信息，则在聚合的时候，需要从第k阶邻居传递信息到k-1阶邻居，并依次传递到中心节点。
- 采样的过程刚好与此相反，在构造第t轮训练的Mini-batch时，我们从中心节点出发，在前序节点集合中采样$N_t$个邻居节点加入采样集合。
- 接着将邻居节点作为新的中心节点继续进行第t-1轮训练的节点采样，以此类推。
- 最后将采样到的节点和边一起构造得到子图。

下面请将GraphSage的采样函数补充完整。

In [11]:
%%writefile userdef_sample.py

import numpy as np

def traverse(item):
    """traverse
    """
    if isinstance(item, list) or isinstance(item, np.ndarray):
        for i in iter(item):
            for j in traverse(i):
                yield j
    else:
        yield item


def flat_node_and_edge(nodes):
    """flat_node_and_edge
    """
    nodes = list(set(traverse(nodes)))
    return nodes


def my_graphsage_sample(graph, batch_train_samples, samples):
    """
    输入：graph - 图结构 Graph
         batch_train_samples - 中心节点 list (batch_size,)
         samples - 采样时的最大邻节点数列表 list 
    输出：被采样节点下标的集合 
         对当前节点进行k阶采样后得到的子图 
    """
    
    start_nodes = batch_train_samples
    nodes = start_nodes
    edges = []
    for max_deg in samples:
        #################################
        # 请在这里补充每阶邻居采样的代码：此部分课堂实践内容已详细讲解，加油~
        # 提示：graph.sample_predecessor(该 API用于获取目标节点对应的源节点，具体用法到 pgl.Graph 结构中查看)
        pred_nodes = graph.sample_predecessor(start_nodes, max_degree = max_deg)
        # 根据采样的子节点， 恢复边
        for dst_node, src_nodes in zip(start_nodes, pred_nodes):
            for node in src_nodes:
                edges.append((node, dst_node))
        #################################

        # 合并已采样节点并找出新的节点作为start_nodes
        last_nodes = nodes
        nodes = [nodes, pred_nodes]
        nodes = flat_node_and_edge(nodes)
        start_nodes = list(set(nodes) - set(last_nodes))
        if len(start_nodes) == 0:
            break

    subgraph = graph.subgraph(
         nodes=nodes,
         edges=edges,
         with_node_feat=False,
         with_edge_feat=False)
         
    return nodes, subgraph

Overwriting userdef_sample.py


运行一下代码看看自己实现的采样算法与PGL相比效果如何吧~ 

In [12]:
!python train.py --use_my_sample

[INFO] 2020-11-27 14:07:46,344 [    train.py:  310]:	Namespace(batch_size=128, epoch=50, graphsage_type='graphsage_maxpool', hidden_size=128, lr=0.01, normalize=False, sample_workers=5, samples_1=25, samples_2=10, symmetry=False, use_cuda=False, use_my_lstm=False, use_my_maxpool=False, use_my_sample=True)
[INFO] 2020-11-27 14:07:47,314 [    train.py:  176]:	preprocess finish
[INFO] 2020-11-27 14:07:47,314 [    train.py:  177]:	Train Examples: 140
[INFO] 2020-11-27 14:07:47,315 [    train.py:  178]:	Val Examples: 300
[INFO] 2020-11-27 14:07:47,315 [    train.py:  179]:	Test Examples: 1000
[INFO] 2020-11-27 14:07:47,315 [    train.py:  180]:	Num nodes 2708
[INFO] 2020-11-27 14:07:47,315 [    train.py:  181]:	Num edges 8137
[INFO] 2020-11-27 14:07:47,315 [    train.py:  182]:	Average Degree 3.00480059084195
[INFO] 2020-11-27 14:07:47,624 [    train.py:  171]:	train Epoch 0 Loss 1.95629 Acc 0.10000 Speed(per batch) 0.09468 sec
[INFO] 2020-11-27 14:07:47,754 [    train.py:  171]:	val Epoch 

## 3. GraphSage聚合函数实现

对于GraphSage中的聚合函数，首先用PGL中的Send和Receive接口实现邻居信息的聚合，然后分别学习两个全连接层，映射得到当前节点和邻居信息的表示，最后将二者拼接起来经过L2标准化，得到新的的节点表示。不同聚合函数的区别就在于信息传递机制的不同。

### 3.1 Mean Aggregator示例代码

以下代码实现了Mean Aggregator的消息传递机制，得到邻居聚合信息后的代码与其他聚合函数相同。具体实现细节可参考第三节实践教程中的消息传递机制。

``` python
def graphsage_mean(gw, feature, hidden_size, act, name):
    # 消息的传递和接收
    def copy_send(src_feat, dst_feat, edge_feat):
    	return src_feat["h"]
    def mean_recv(feat):
    	return fluid.layers.sequence_pool(feat, pool_type="average")
    msg = gw.send(copy_send, nfeat_list=[("h", feature)])
    neigh_feature = gw.recv(msg, mean_recv)
    
    # 自身表示和邻居表示的结合
    self_feature = feature
    self_feature = fluid.layers.fc(self_feature,
                                   hidden_size,
                                   act=act,
                                   name=name + '_l')
    neigh_feature = fluid.layers.fc(neigh_feature,
                                    hidden_size,
                                    act=act,
                                    name=name + '_r')
    output = fluid.layers.concat([self_feature, neigh_feature], axis=1)
    output = fluid.layers.l2_normalize(output, axis=1)
    return output
```

### 3.2 MaxPool Aggregator实现

MaxPool Aggregator在进行邻居聚合时会选取最大的值作为当前节点接收到的消息，实现API可参考[Paddle文档](https://www.paddlepaddle.org.cn/documentation/docs/zh/api_cn/layers_cn.html)。

实际实现的时候，与上述给出的例子 Mean Aggregator 非常类似。大家可以自行填空完成。

In [ ]:
%%writefile userdef_maxpool.py
import paddle.fluid as fluid

def my_graphsage_maxpool(gw,
                      feature,
                      hidden_size,
                      act,
                      name,
                      inner_hidden_size=512):
    """
    输入：gw - GraphWrapper对象
         feature - 当前节点表示 (num_nodes, embed_dim)
         hidden_size - 新的节点表示维数 int
         act - 激活函数名 str
         name - 聚合函数名 str
         inner_hidden_size - 消息传递过程中邻居信息的维数 int
    输出：新的节点表示
    """
    
    ####################################
    # 请在这里实现MaxPool Aggregator

    def copy_send(src_feat, dst_feat, edge_feat):
          return src_feat["h"]
    def maxpool_recv(feat):
          return fluid.layers.sequence_pool(feat, pool_type="max")

    # 求和聚合函数
    def sumpool_recv(feat):
          return fluid.layers.sequence_pool(feat, pool_type="sum")

    # 补充消息传递机制触发代码
    neigh_feature = fluid.layers.fc(feature, inner_hidden_size, act="relu")
    msg = gw.send(copy_send, nfeat_list=[("h", neigh_feature)])
    neigh_feature = gw.recv(msg, maxpool_recv)
    ####################################
    
    # 自身表示和邻居表示的结合
    self_feature = feature
    self_feature = fluid.layers.fc(self_feature,
                                   hidden_size,
                                   act=act,
                                   name=name + '_l')
    neigh_feature = fluid.layers.fc(neigh_feature,
                                    hidden_size,
                                    act=act,
                                    name=name + '_r')
    output = fluid.layers.concat([self_feature, neigh_feature], axis=1)
    output = fluid.layers.l2_normalize(output, axis=1)
    return output

In [9]:
!python train.py --use_my_maxpool

[INFO] 2020-11-27 14:02:42,444 [    train.py:  310]:	Namespace(batch_size=128, epoch=50, graphsage_type='graphsage_maxpool', hidden_size=128, lr=0.01, normalize=False, sample_workers=5, samples_1=25, samples_2=10, symmetry=False, use_cuda=False, use_my_lstm=False, use_my_maxpool=True, use_my_sample=False)
[INFO] 2020-11-27 14:02:43,420 [    train.py:  176]:	preprocess finish
[INFO] 2020-11-27 14:02:43,420 [    train.py:  177]:	Train Examples: 140
[INFO] 2020-11-27 14:02:43,420 [    train.py:  178]:	Val Examples: 300
[INFO] 2020-11-27 14:02:43,420 [    train.py:  179]:	Test Examples: 1000
[INFO] 2020-11-27 14:02:43,420 [    train.py:  180]:	Num nodes 2708
[INFO] 2020-11-27 14:02:43,420 [    train.py:  181]:	Num edges 8137
[INFO] 2020-11-27 14:02:43,420 [    train.py:  182]:	Average Degree 3.00480059084195
[INFO] 2020-11-27 14:02:43,726 [    train.py:  171]:	train Epoch 0 Loss 1.90015 Acc 0.25714 Speed(per batch) 0.09209 sec
[INFO] 2020-11-27 14:02:43,857 [    train.py:  171]:	val Epoch 

***此截图测试sum聚合函数***
![](https://ai-studio-static-online.cdn.bcebos.com/311c749f9a664de08f752be49e12157084675bb611a940408891663dd5f79aaa)


***lstm聚合函数测试***

In [24]:
%%writefile userdef_lstm.py
import paddle.fluid as fluid

def my_graphsage_lstm(gw,
                      feature,
                      hidden_size,
                      act,
                      name,
                      inner_hidden_size=128):
    """
    输入：gw - GraphWrapper对象
         feature - 当前节点表示 (num_nodes, embed_dim)
         hidden_size - 新的节点表示维数 int
         act - 激活函数名 str
         name - 聚合函数名 str
         inner_hidden_size - 消息传递过程中邻居信息的维数 int
    输出：新的节点表示
    """
    def copy_send(src_feat, dst_feat, edge_feat):
          return src_feat["h"]
    def lstmpool_recv(feat):
        hidden_dim = 128
        forward, _ = fluid.layers.dynamic_lstm(
            input=feat, size=hidden_dim * 4, use_peepholes=False)
        output = fluid.layers.sequence_last_step(forward)
        return output

    hidden_dim=128
    # 补充消息传递机制触发代码
    neigh_feature = fluid.layers.fc(feature, inner_hidden_size, act="relu")
    forward_proj = fluid.layers.fc(input=neigh_feature,size=hidden_dim * 4, bias_attr=False,name="lstm_proj")
    msg = gw.send(copy_send, nfeat_list=[("h", forward_proj)])
    neigh_feature = gw.recv(msg, lstmpool_recv)
    ####################################
    
    # 自身表示和邻居表示的结合
    self_feature = feature
    self_feature = fluid.layers.fc(self_feature,
                                   hidden_size,
                                   act=act,
                                   name=name + '_l')
    neigh_feature = fluid.layers.fc(neigh_feature,
                                    hidden_size,
                                    act=act,
                                    name=name + '_r')
    output = fluid.layers.concat([self_feature, neigh_feature], axis=1)
    output = fluid.layers.l2_normalize(output, axis=1)
    return output

Overwriting userdef_lstm.py


In [25]:
!python train.py --use_my_lstm

[INFO] 2020-11-27 14:46:30,832 [    train.py:  310]:	Namespace(batch_size=128, epoch=50, graphsage_type='graphsage_maxpool', hidden_size=128, lr=0.01, normalize=False, sample_workers=5, samples_1=25, samples_2=10, symmetry=False, use_cuda=False, use_my_lstm=True, use_my_maxpool=False, use_my_sample=False)
[INFO] 2020-11-27 14:46:31,787 [    train.py:  176]:	preprocess finish
[INFO] 2020-11-27 14:46:31,787 [    train.py:  177]:	Train Examples: 140
[INFO] 2020-11-27 14:46:31,787 [    train.py:  178]:	Val Examples: 300
[INFO] 2020-11-27 14:46:31,787 [    train.py:  179]:	Test Examples: 1000
[INFO] 2020-11-27 14:46:31,787 [    train.py:  180]:	Num nodes 2708
[INFO] 2020-11-27 14:46:31,787 [    train.py:  181]:	Num edges 8137
[INFO] 2020-11-27 14:46:31,787 [    train.py:  182]:	Average Degree 3.00480059084195
[INFO] 2020-11-27 14:46:32,111 [    train.py:  171]:	train Epoch 0 Loss 1.95309 Acc 0.15714 Speed(per batch) 0.09531 sec
[INFO] 2020-11-27 14:46:32,241 [    train.py:  171]:	val Epoch 

***这个我是按照examples里面的代码添加的，不知道lstm聚合为什么效果不是很好........***

其实在 GraphSage 原文中，还提出可以使用 LSTM 进行聚合。由于LSTM的输入是有序的而节点的邻居是无序的，论文将邻居节点随机排列作为LSTM的输入。这里我们就不做作业要求了，感兴趣的同学可以查看对应的[代码](https://github.com/PaddlePaddle/PGL/blob/main/examples/graphsage/model.py)。